In [1]:
import os
from datetime import datetime

import pandas as pd
from feast import FeatureStore

/opt/vscode/otus-feature-store/.venv/lib/python3.11/site-packages/feast/permissions/auth_model.py:35: PydanticDeprecatedSince212: Using `@model_validator` with mode='after' on a classmethod is deprecated. Instead, use an instance method. See the documentation at https://docs.pydantic.dev/2.12/concepts/validators/#model-after-validator. Deprecated in Pydantic V2.12 to be removed in V3.0.
  @model_validator(mode="after")


In [2]:
raw_data_path = os.path.join("feature_store", "feature_repo", "data", "driver_stats.parquet")
feature_store_path = os.path.join("feature_store", "feature_repo")

### Check data

In [3]:
df = pd.read_parquet(raw_data_path)

In [4]:
df.head(5)

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-10-17 12:07:08.228578+00:00,1001,1.000000,1.000000,1000,2024-10-17 12:07:08.228581
1,2024-10-02 11:00:00+00:00,1005,0.429879,0.194598,582,2024-10-17 11:30:07.072000
2,2024-10-02 12:00:00+00:00,1005,0.230119,0.642878,551,2024-10-17 11:30:07.072000
3,2024-10-02 13:00:00+00:00,1005,0.128600,0.674187,38,2024-10-17 11:30:07.072000
4,2024-10-02 14:00:00+00:00,1005,0.400603,0.473636,583,2024-10-17 11:30:07.072000


### Features inference

In [5]:
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
    }
)

In [6]:
entity_df

,driver_id,event_timestamp,label_driver_reported_satisfaction
0,1001,2021-04-12 10:59:42,1
1,1002,2021-04-12 08:12:10,5
2,1003,2021-04-12 16:40:26,3


In [7]:
store = FeatureStore(repo_path=feature_store_path)

/opt/vscode/otus-feature-store/.venv/lib/python3.11/site-packages/feast/repo_config.py:291: DeprecationWarning: The serialization version below 3 are deprecated. Specifying `entity_key_serialization_version` to 3 is recommended.
  warnings.warn(


In [8]:
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
).to_df()

print("----- Feature schema -----\n")
print(training_df.info())

----- Feature schema -----

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   driver_id                           3 non-null      int64              
 1   event_timestamp                     3 non-null      datetime64[ns, UTC]
 2   label_driver_reported_satisfaction  3 non-null      int64              
 3   conv_rate                           3 non-null      float32            
 4   acc_rate                            3 non-null      float32            
 5   avg_daily_trips                     3 non-null      int32              
dtypes: datetime64[ns, UTC](1), float32(2), int32(1), int64(2)
memory usage: 240.0 bytes
None


In [9]:
training_df.head()

,driver_id,event_timestamp,label_driver_reported_satisfaction,conv_rate,acc_rate,avg_daily_trips
0,1001,2021-04-12 10:59:42+00:00,1,0.709758,0.692957,402
1,1002,2021-04-12 08:12:10+00:00,5,0.718295,0.584081,370
2,1003,2021-04-12 16:40:26+00:00,3,0.697411,0.197680,25


### Features View - on demand

In [10]:
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

In [11]:
entity_df.head(5)

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2
0,1001,2021-04-12 10:59:42,1,1,10
1,1002,2021-04-12 08:12:10,5,2,20
2,1003,2021-04-12 16:40:26,3,3,30


In [12]:
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

In [13]:
training_df

,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2021-04-12 10:59:42+00:00,1,1,10,0.709758,0.692957,402,1.709758,10.709758
1,1002,2021-04-12 08:12:10+00:00,5,2,20,0.718295,0.584081,370,2.718295,20.718295
2,1003,2021-04-12 16:40:26+00:00,3,3,30,0.697411,0.197680,25,3.697411,30.697411


In [14]:
# Online feature retrieval
online_features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[{"driver_id": 1001}, {"driver_id": 1002}],
).to_dict()

print("Online features for drivers 1001, 1002:")
for key, value in online_features.items():
    print(f"{key}: {value}")

Online features for drivers 1001, 1002:
driver_id: [1001, 1002]
acc_rate: [None, None]
conv_rate: [None, None]
avg_daily_trips: [None, None]


/opt/vscode/otus-feature-store/.venv/lib/python3.11/site-packages/feast/infra/key_encoding_utils.py:141: UserWarning: Serialization of entity key with version < 3 is removed. Please use version 3 by setting entity_key_serialization_version=3.To reserializa your online store featrues refer -  https://github.com/feast-dev/feast/blob/master/docs/how-to-guides/entity-reserialization-of-from-v2-to-v3.md
  warnings.warn(


In [15]:
# Using Feature Service for consistent feature sets
training_df_v1 = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("driver_activity_v1")
).to_df()

print("\nFeatures from driver_activity_v1 service:")
training_df_v1.head()


Features from driver_activity_v1 service:


,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2021-04-12 10:59:42+00:00,1,1,10,0.709758,1.709758,10.709758
1,1002,2021-04-12 08:12:10+00:00,5,2,20,0.718295,2.718295,20.718295
2,1003,2021-04-12 16:40:26+00:00,3,3,30,0.697411,3.697411,30.697411


In [16]:
# Get feature view metadata
feature_view = store.get_feature_view("driver_hourly_stats")
print("\nFeature view metadata:")
print(f"Name: {feature_view.name}")
print(f"Entities: {feature_view.entities}")
print(f"TTL: {feature_view.ttl}")
print(f"Online: {feature_view.online}")
print(f"Features: {[f.name for f in feature_view.features]}")


Feature view metadata:
Name: driver_hourly_stats
Entities: ['driver']
TTL: 1 day, 0:00:00
Online: True
Features: ['conv_rate', 'acc_rate', 'avg_daily_trips']


In [17]:
# Using Feature Service for consistent feature sets
training_df_v4 = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("driver_activity_v4")
).to_df()

print("\nFeatures from driver_activity_v4 service:")
training_df_v4.head()


Features from driver_activity_v4 service:


,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,combined_rating,performance_score
0,1001,2021-04-12 10:59:42+00:00,1,1,10,0.709758,0.692957,402,0.703038,4.217479
1,1002,2021-04-12 08:12:10+00:00,5,2,20,0.718295,0.584081,370,0.664610,3.931966
2,1003,2021-04-12 16:40:26+00:00,3,3,30,0.697411,0.197680,25,0.497519,1.620965


In [21]:
#  Данные для исторического запроса (обучение)
entity_df = pd.DataFrame({
    "driver_id": [1001, 1002, 1003],
    "event_timestamp": [
        pd.Timestamp("2021-04-12 10:59:42", tz="UTC"),
        pd.Timestamp("2021-04-12 08:12:10", tz="UTC"),
        pd.Timestamp("2021-04-12 16:40:26", tz="UTC"),
    ],
})

# Получаем исторические признаки
historical_features = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_reliability:conv_rate",
        "driver_reliability:acc_rate",
        "driver_activity:avg_daily_trips",
        "driver_risk_adjusted_performance:risk_adjusted_score",
        "driver_risk_adjusted_performance:engagement_level",
    ]
).to_df()

# Выводим таблицу
print(" Исторические признаки (для обучения):")
display(historical_features[[
    "driver_id", "event_timestamp",
    "conv_rate", "acc_rate", "avg_daily_trips",
    "risk_adjusted_score", "engagement_level"
]])

#  Онлайн-инференс (реальное время)
# Получаем признаки без временной привязки — только по driver_id
online_response = store.get_online_features(
    features=[
        "driver_reliability:conv_rate",
        "driver_reliability:acc_rate",
        "driver_activity:avg_daily_trips",
        "driver_risk_adjusted_performance:risk_adjusted_score",
        "driver_risk_adjusted_performance:engagement_level",
    ],
    entity_rows=[
        {"driver_id": 1001},
        {"driver_id": 1002},
        {"driver_id": 1003},
    ]
).to_dict()

# Преобразуем в удобный DataFrame для отображения
online_df = pd.DataFrame(online_response)
print("\n⚡ Онлайн-признаки (для инференса):")
display(online_df[[
    "driver_id",
    "conv_rate", "acc_rate", "avg_daily_trips",
    "risk_adjusted_score", "engagement_level"
]])

 Исторические признаки (для обучения):


,driver_id,event_timestamp,conv_rate,acc_rate,avg_daily_trips,risk_adjusted_score,engagement_level
0,1001,2021-04-12 10:59:42+00:00,0.709758,0.692957,402,3.579146,1.255887
1,1002,2021-04-12 08:12:10+00:00,0.718295,0.584081,370,3.180653,1.173241
2,1003,2021-04-12 16:40:26+00:00,0.697411,0.197680,25,1.055368,0.786989



⚡ Онлайн-признаки (для инференса):


/opt/vscode/otus-feature-store/.venv/lib/python3.11/site-packages/feast/infra/key_encoding_utils.py:141: UserWarning: Serialization of entity key with version < 3 is removed. Please use version 3 by setting entity_key_serialization_version=3.To reserializa your online store featrues refer -  https://github.com/feast-dev/feast/blob/master/docs/how-to-guides/entity-reserialization-of-from-v2-to-v3.md
  warnings.warn(


,driver_id,conv_rate,acc_rate,avg_daily_trips,risk_adjusted_score,engagement_level
0,1001,None,None,None,0.0,0.000001
1,1002,None,None,None,0.0,0.000001
2,1003,None,None,None,0.0,0.000001
